# Learning Goals
In Assignment 2, we learnt how to construct networks of spiking neurons and propagate information through a network of fixed weights. In this assignment, you will learn how to train network weights for a given task using brain-inspired learning rules.

Let's import all the libraries required for this assignment.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Question 1: Training a Network

## 1a.
What is the purpose of a learning algorithm? In other words, what does a learning algorithm dictate, and what is the objective of it?

## Answer 1a.
The purpose of a learning algorithm is to push information from data into the structure of the network. The learning algorithm dictates "how" that information is computed by modulating parameters in the network. The objective function is usually a form of "loss minimization" (i.e. cross entropy loss). The overall goal is to build a suitable model during training which is generalizable to unseen data during testing.

## 1b.
Categorize and explain the various learning algorithms w.r.t. biological plausibility. Can you explain the tradeoffs involved with the different learning rules? *Hint: Think computational advantages and disadvantages of biological plausibility.*

## Answer 1b.

- High Biological Plausability: Hebbian Learning
  - "Neurons that fire together wire together" which mirrors synaptic plasticity in biology. This is a simple learning rule, that explains unsupervised feature detection and pattern recognition. But, this approach fails with complicated temporal sequences.
- High Biological Plausability: Spike-Timing Dependent Plasticity
  - We have the Hebbian Learning idea but we also add in the idea of timing the neuronal spikes appropriately, which is more biologically plausible. We can get temporal dynamics at the cost of increased computational cost.
- Moderate Biological Plausibility: Reinforcement Learning
  - Reward prediction error is similar to dopaminergic signaling in the brain. RL captures agentic behavior as is classical in a MDP; however, RL is computationally complex, and may not capture full complexity of biological systems in smaller, simplified networks.
- Low Biological Plausability: Backpropogation
  - Learns by using error signal and propogates that through the network via the chain rule in calculus. This works well for industry-scale models; however, it's not biologically plausible (i.e. brain uses local updates but backprop uses a global update signal, backprop uses both a forward and backward pass which is not observed biologically).

  Generally speaking, more biologically plausible means more computationally complex and less biologically plausible means less computationally complex.



# Question 2: Hebbian Learning

## 2a.

In this exercise, you will implement the hebbian learning rule to solve AND Gate. First, we need to create a helper function to generate the training data. The function should return lists of tuples where each tuple comprises of numpy arrays of rate-coded inputs and the corresponding rate-coded output.

Below is the function to generate the training data. Fill the components to return the training data.

In [2]:
def genANDTrainData(snn_timestep):
    """
    Function to generate the training data for AND
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output

        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """

    #Initialize an empty list for train data
    train_data = []

    #encode 0. Numpy random choice function might be useful here.
    encode_0 = np.random.choice([0, 1], size=snn_timestep, p=[0.90, 0.10])

    #encode 1. Numpy random choice function might be useful here.
    encode_1 = np.random.choice([0, 1], size=snn_timestep, p=[0.10, 0.90])

    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here.
    train_data.append((np.stack((encode_0, encode_0)), np.zeros(snn_timestep)))
    train_data.append((np.stack((encode_0, encode_1)), np.zeros(snn_timestep)))
    train_data.append((np.stack((encode_1, encode_0)), np.zeros(snn_timestep)))
    train_data.append((np.stack((encode_1, encode_1)), np.ones(snn_timestep)))

    return train_data

## 2b.
We will use the implementation of the network from assignment 2 to create an SNN comprising of one input layer and one output layer. Can you explain algorithmically, how you can use this simple architecture to learn AND gate. Your algorithm should comprise of encoding, forward propagation, network training, and decoding steps.

## Answer 2b.

Step 1 - Encoding:
Generate training data $\mathcal{D}$ for $\{(0,0), (0,1), (1,0), (1,1)\}$ using `genANDTrainData` function in 2A. This function gives us Spike In and Spike Out representations for all possible two-sized tuple binary combinations.

Step 2 - Forward Propogation: Set up `SNN` object, initialized with an input dimension of 2 and an output dimension of 1. Also, set up a random `input_2_output_weight` matrix to be later updated during training.

Step 3 - Network Training: Compute output neuron average firing rate and compare to input neuron firing rate. Increase the weights with the Hebbian Learning Rule if firing rates correspond.

Step 4 - Decoding: Test the trained `SNN` model. After a model is encoded and passed through the model, aggregate output spikes to compute the firing rate and see how it compares to the average firing rate output neuron threshold.


The SNN has already been implemented for you. You do not need to do anything here. Just understand the implementation so that you can use it in the later parts.

In [3]:
class LIFNeurons:
    """
        Define Leaky Integrate-and-Fire Neuron Layer
        This class is complete. You do not need to do anything here.
    """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons

        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)

    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic inputs
        Return:
            self.spike: output spikes from the layer
                """
        self.volt = self.vdecay * self.volt * (1. - self.spike) + psp_input
        self.spike = (self.volt > self.vth).astype(float)
        return self.spike

class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            weights (ndarray): connection weights
            pre_dimension (int): dimension for pre-synaptic neurons
            post_dimension (int): dimension for post-synaptic neurons
        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension

    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: postsynaptic layer activations
        """
        psp = np.matmul(self.weights, spike_input)
        return psp


class SNN:
    """ Define a Spiking Neural Network with No Hidden Layer """

    def __init__(self, input_2_output_weight,
                 input_dimension=2, output_dimension=2,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer
            input_dimension (int): input dimension
            hidden_dimension (int): hidden_dimension
            output_dimension (int): output_dimension
            vdecay (float): voltage decay of LIF neuron
            vth (float): voltage threshold of LIF neuron
            snn_timestep (int): number of timesteps for inference
        """
        self.snn_timestep = snn_timestep
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        self.input_2_output_connection = Connections(input_2_output_weight, input_dimension, output_dimension)

    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            spike outputs of the network
        """
        spike_output = np.zeros(self.output_layer.dimension)
        for tt in range(self.snn_timestep):
            input_2_output_psp = self.input_2_output_connection(spike_encoding[:, tt])
            output_spikes = self.output_layer(input_2_output_psp)
            spike_output += output_spikes
        return spike_output/self.snn_timestep

## 2c.
Next, you need to write a function for network training using hebbian learning rule. The function is defined below. You need to fill in the components so that the network weights are updated in the right manner.

In [8]:
def hebbian(network, train_data, lr=1e-5, epochs=10):
    """
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.

        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights.
    """

    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        for data in train_data:

            input_spikes, _ = data

            #compute the firing rate for the input
            input_fr = np.mean(input_spikes, axis=1)

            #compute the firing rate for the output
            output_fr = np.mean(network(input_spikes))

            #compute the correlation using the firing rates calculated above
            correlation = input_fr * output_fr

            #compute the weight increment
            weight_increment = lr * correlation

            #increment the weight
            network.input_2_output_connection.weights += weight_increment

## 2d.
In this exercise, you will use your implementations above to train an SNN to learn AND gate.

In [9]:
np.random.seed(3)

#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
v_decay = 0.50

#Define a variable for voltage threshold
vth = 0.80

#Define a variable for snn timesteps
snn_timesteps = 50

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension,input_dimension)

#print the initial weights
print("Initial Weights:")
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(
    input_2_output_weight,
    input_dimension,
    output_dimension,
    v_decay,
    vth,
    snn_timesteps
)

#Get the training data for AND gate using the function defined in 2a.
train_data = genANDTrainData(snn_timesteps)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(
    snn,
    train_data,
    lr=1e-3,
    epochs=10
)

#Test the trained network and print the network output for all 4 cases.
print("\nNetwork outputs after training:")
for data in train_data:
    input_spikes, target_output = data
    output = snn(input_spikes)
    print(f"Target: {target_output.mean()}, Predicted Output: {np.round(output)}, Actual Output: {output}")

#Print Final Network Weights
print("Initial Weights:")
print(snn.input_2_output_connection.weights)

Initial Weights:
[[0.5507979  0.70814782]]

Network outputs after training:
Target: 0.0, Predicted Output: [0.], Actual Output: [0.06]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.46]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.48]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]
Initial Weights:
[[0.5639899  0.72116782]]


# Question 3: Limitations of Hebbian Learning rule

## 3a.
Can you learn the AND gate using 2 neurons in the output layer instead of one? If yes, describe what changes you might need to make to your algorithm in 2b. If not, explain why not, and what consequences it might entail for the use of hebbian learning for complex real-world tasks.

## Answer 3a.
Yes, you can.
- Step 1 - Encoding: Keep this the same.
- Step 2 - Forward Propogation: Make `output_dimension=2` and size(`input_2_output_weight`)=`(2,2)`.
- Step 3 - Network Training: There will be two neuron outputs. One neuron should fire for the `encoded(1,1)` case and for all other cases, the other neuron should fire. We can do this by making the correlation rule more fine-grained. We can increase weights for first output neuron if the input is (1,1) and increase weights for second output neuron if the input is anything else.
- Step 4 - Decoding: If the 1st neuron fires, and the 2nd output doesn't then we predict 1 for the (1,1) case. In all other cases, we predict 0.

## 3b.
Train the network using hebbian learning for AND gate with the same arguments as defined in 2d. but now multiply the number of epochs by 20. Can your network still learn AND gate correctly? Inspect the initial and final network weights, and compare them against the network weights in 2d. Based on this, explain your observations for the network behavior.

In [11]:
#Implementation for 3b. (same as 2d. but with change of one argument)

np.random.seed(3)

#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
v_decay = 0.50

#Define a variable for voltage threshold
vth = 0.80

#Define a variable for snn timesteps
snn_timesteps = 50

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension,input_dimension)

#print the initial weights
print("Initial Weights:")
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(
    input_2_output_weight,
    input_dimension,
    output_dimension,
    v_decay,
    vth,
    snn_timesteps
)

#Get the training data for AND gate using the function defined in 2a.
train_data = genANDTrainData(snn_timesteps)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(
    snn,
    train_data,
    lr=1e-3,
    epochs=10*20
)

#Test the trained network and print the network output for all 4 cases.
print("\nNetwork outputs after training:")
for data in train_data:
    input_spikes, target_output = data
    output = snn(input_spikes)
    print(f"Target: {target_output.mean()}, Predicted Output: {np.round(output)}, Actual Output: {output}")

#Print Final Network Weights
print("Initial Weights:")
print(snn.input_2_output_connection.weights)

Initial Weights:
[[0.5507979  0.70814782]]

Network outputs after training:
Target: 0.0, Predicted Output: [0.], Actual Output: [0.06]
Target: 0.0, Predicted Output: [1.], Actual Output: [0.92]
Target: 0.0, Predicted Output: [1.], Actual Output: [0.92]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]
Initial Weights:
[[0.8204799  1.02328942]]


## Answer 3b.
No, with 20 times as many epochs, the network cannot still learn the AND gate correctly. We have a few observations:
- Initial Weights: `[[0.5507979  0.70814782]]`
- Final Weights (10 Epochs): `[[0.5639899  0.72116782]]`
- Final Weights (10*20 Epochs): `[[0.8204799  1.02328942]]`

We notice that the simplest Hebbian Learning rule updates the weight as $w_{new}= w_{old} + \Delta w$, where $\Delta w = xy$, the correlation between the input and the output. But notice, that this term $\Delta w$ is always non-negative. Therefore, as we increase the number of epochs, the weights also correspondingly increase without bound, towards $\infty$. Hence, after a certain number of epochs, the simplest Hebbian Learning rule is not meaningful in learning structure. The appropriate next question is, how can we bound the updates?

## 3c.
Based on your observations and response in 3b., can you explain another limitation of hebbian learning rule w.r.t. weight growth? Can you also suggest a possible remedy for it?

## Answer 3c.
The simplest Hebbian Learning rule does not have a mechanism for forgetting or weight decay. Currently, the weights can only increase or remain the same based on the input/output firing rate correlations. This is an issue because the learning mechanism is not dynamic, and may update the weights due to spurious correlations between input/output firing rates. A solution might be to decay the current weights by some factor $\alpha$. So instead of having the update rule be $\Delta w = \eta \cdot xy$, we can have a decay mechanism with $\Delta w = \eta \cdot xy - \alpha \cdot y^2 w$

## 3d.
To resolve the issues with hebbian learning, one possibility is Oja's rule. In this exercise, you will implement and train an SNN using Oja's learning rule.

In [12]:
def oja(network, train_data, lr=1e-5, epochs=10):
    """
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.

        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights.
    """

    #iterate over the epochs
    for ee in range(epochs):
        #iterate over all samples in train_data
        for data in train_data:

            input_spikes, _ = data

            #compute the firing rate for the input
            input_fr = np.mean(input_spikes, axis=1)

            #compute the firing rate for the output
            output_fr = np.mean(network(input_spikes))

            #compute the weight increment
            correlation = input_fr * output_fr
            weight_increment = lr * (correlation - output_fr**2 * network.input_2_output_connection.weights)

            #increment the weight
            network.input_2_output_connection.weights += weight_increment

Now, test your implementation below.

In [13]:
np.random.seed(3)

#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
v_decay = 0.50

#Define a variable for voltage threshold
vth = 0.80

#Define a variable for snn timesteps
snn_timesteps=50

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension)

#print the initial weights
print("Initial Weights:")
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight,
          input_dimension,
          output_dimension,
          v_decay,
          vth,
          snn_timesteps)

#Get the training data for AND gate using the function defined in 2a.
train_data = genANDTrainData(snn_timesteps)

#Train the network using the function defined in 3d. with the appropriate arguments
oja(
    snn,
    train_data,
    lr=1e-3,
    epochs=10*20
)

#Test the trained network and print the network output for all 4 cases.
print("\nNetwork outputs after training:")
for data in train_data:
    input_spikes, target_output = data
    output = snn(input_spikes)
    print(f"Target: {target_output.mean()}, Predicted Output: {np.round(output)}, Actual Output: {output}")

#Print Final Network Weights
print("Initial Weights:")
print(snn.input_2_output_connection.weights)

Initial Weights:
[[0.5507979  0.70814782]]

Network outputs after training:
Target: 0.0, Predicted Output: [0.], Actual Output: [0.06]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.46]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.48]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]
Initial Weights:
[[0.65791595 0.77637841]]


# Question 4: Spike-time dependent plasticity (STDP)

## 4a.
What is the limitation with hebbian learning that STDP aims to resolve?

## Answer 4a.
Hebbian learning updates synaptic weights based on correlations between pre-synaptic and post-synaptic activity. However, these updates don't take into account timing (i.e. causation) between presynaptic firing and postsynaptic firing. STDP strengthens connections when presynaptic firing precedes postsynaptic firing and weakens them if postsynaptic firing precedes presynaptic firing.

## 4b.
Describe the algorithm to train a network using STDP learning rule. You do not need to describe encoding here. Your algorithm should be such that its naturally translatable to a program.

## Answer 4b.
Parameters: $A^{+}$ and $A^{-}$ are the amplitudes of potentiation and depression. $\tau_{+}$ and $\tau_{-}$ are the time constants for potentiation and depression. $w_{min}$ and $w_{max}$ make sure that the updates are clipped and bound within the range $[w_{min}, w_{max}]$.

1. Record all the spike times in neurons $(i, j)$ as $t_{i}$ and $t_{j}$
2. Compute $\Delta t = t_{j} - t_{i}$ for all spike times, which is $t$
3. Update the weights as $w_{new} = \begin{cases} \max(w_{min}, w_{old} + \eta \cdot \Delta w) & \text{if } \Delta w > 0 \\ \min(w_{max}, w_{old} + \eta \cdot \Delta w) & \text{if } \Delta w < 0 \end{cases}$
4. Then, update the weights as $w_{new} = w_{old} + \eta \cdot \Delta w \cdot b$
5.Repeat this for all neuron pairs $i$ for $k$ epochs.

## 4c.
In this exercise, you will implement the STDP learning algorithm to train a network. STDP has many different flavors. For this exercise, we will use the learning rule defined in: https://dl.acm.org/doi/pdf/10.1609/aaai.v33i01.330110021. Pay special attention to Equations 2 and 3.

Below is the class definition for STDP learning algorithm. Your task is to fill in the components so that the weights are updated in the right manner.

In [14]:
class STDP():
    """Train a network using STDP learning rule"""
    def __init__(self, network, A_plus, A_minus, tau_plus, tau_minus, lr, snn_timesteps=20, epochs=30, w_min=0, w_max=1):
        """
        Args:
            network (SNN): network which needs to be trained
            A_plus (float): STDP hyperparameter
            A_minus (float): STDP hyperparameter
            tau_plus (float): STDP hyperparameter
            tau_minus (float): STDP hyperparameter
            lr (float): learning rate
            snn_timesteps (int): SNN simulation timesteps
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.
            w_min (float): lower bound for the weights
            w_max (float): upper bound for the weights
        """
        self.network = network
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus
        self.snn_timesteps = snn_timesteps
        self.lr = lr
        self.time = np.arange(0, self.snn_timesteps, 1)
        self.sliding_window = np.arange(-4, 4, 1) #defines a sliding window for STDP operation.
        self.epochs = epochs
        self.w_min = w_min
        self.w_max = w_max

    def update_weights(self, t, i):
        """
        Function to update the network weights using STDP learning rule

        Args:
            t (int): time difference between postsynaptic spike and a presynaptic spike in a sliding window
            i(int): index of the presynaptic neuron

        Fill the details of STDP implementation
        """
        #compute delta_w for positive time difference
        if t>0:
          delta_w = self.A_plus*np.exp(-t/self.tau_plus)

        #compute delta_w for negative time difference
        else:
          delta_w = -self.A_minus*np.exp(-t/self.tau_minus)

        #update the network weights if weight increment is negative
        if delta_w < 0:
          self.network.input_2_output_connection.weights[:, i] = np.maximum(self.w_min, self.network.input_2_output_connection.weights[:, i] + self.lr * delta_w)

        #update the network weights if weight increment is positive
        elif delta_w > 0:
          self.network.input_2_output_connection.weights[:, i] = np.minimum(self.w_max, self.network.input_2_output_connection.weights[:, i] + self.lr * delta_w)

    def train_step(self, train_data_sample):
        """
        Function to train the network for one training sample using the update function defined above.

        Args:
            train_data_sample (list): a sample from the training data

        This function is complete. You do not need to do anything here.
        """
        input = train_data_sample[0]
        output = train_data_sample[1]
        for t in self.time:
            if output[t] == 1:
                for i in range(2):
                    for t1 in self.sliding_window:
                        if (0<= t + t1 < self.snn_timesteps) and (t1!=0) and (input[i][t+t1] == 1):
                            self.update_weights(t1, i)

    def train(self, training_data):
        """
        Function to train the network

        Args:
            training_data (list): training data

        This function is complete. You do not need to do anything here.
        """
        for ee in range(self.epochs):
            for train_data_sample in training_data:
                self.train_step(train_data_sample)

Let's test the implementation

In [16]:
np.random.seed(3)

#Define a variable for input dimension
input_dimension=2

#Define a variable for output dimension
output_dimension=1

#Define a variable for voltage decay
v_decay=0.5

#Define a variable for voltage threshold
vth=0.8

#Define a variable for snn timesteps
snn_timesteps=50

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension)
print("Initial Weights:")
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight,
          input_dimension,
          output_dimension,
          v_decay,
          vth,
          snn_timesteps)

#Get the training data for AND gate using the function defined in 2a.
train_data = genANDTrainData(snn_timesteps)

#Create an object of STDP class with appropriate arguments
stdp = STDP(
    network=snn,
    A_plus = 0.10,
    A_minus = 0.10,
    tau_plus = 20,
    tau_minus=20,
    lr=1e-5,
    snn_timesteps=snn_timesteps,
    epochs=10*20,
    w_min=0,
    w_max=1,
)

#Train the network using STDP
stdp.train(train_data)

#Test the trained network and print the network output for all 4 cases.
print("\nNetwork outputs after training:")
for data in train_data:
    input_spikes, target_output = data
    output = snn(input_spikes)
    print(f"Target: {target_output.mean()}, Predicted Output: {np.round(output)}, Actual Output: {output}")
print("\nFinal weights:")
print(snn.input_2_output_connection.weights)

Initial Weights:
[[0.5507979  0.70814782]]

Network outputs after training:
Target: 0.0, Predicted Output: [0.], Actual Output: [0.06]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.46]
Target: 0.0, Predicted Output: [0.], Actual Output: [0.48]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]

Final weights:
[[0.53563823 0.69298815]]


# Question 5: OR Gate
Can you train the network with the same architecture in Q2-4 for learning the OR gate. You will need to create another function called genORTrainData. Then create an SNN and train it using STDP.

In [17]:
#Write your implementation of genORTrainData here.

def genORTrainData(snn_timestep):
    """
    Function to generate the training data for AND
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output

        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """

    #Initialize an empty list for train data
    train_data = []

    #encode 0. Numpy random choice function might be useful here.
    encode_0 = np.random.choice([0, 1], size=snn_timestep, p=[0.90, 0.10])

    #encode 1. Numpy random choice function might be useful here.
    encode_1 = np.random.choice([0, 1], size=snn_timestep, p=[0.10, 0.90])

    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here.
    train_data.append((np.stack((encode_0, encode_0)), np.zeros(snn_timestep)))
    train_data.append((np.stack((encode_0, encode_1)), np.ones(snn_timestep)))
    train_data.append((np.stack((encode_1, encode_0)), np.ones(snn_timestep)))
    train_data.append((np.stack((encode_1, encode_1)), np.ones(snn_timestep)))

    return train_data

In [19]:
#Train the network for OR gate here using the implementation from 4c.

np.random.seed(3)

#Define a variable for input dimension
input_dimension=2

#Define a variable for output dimension
output_dimension=1

#Define a variable for voltage decay
v_decay=0.5

#Define a variable for voltage threshold
vth=0.5

#Define a variable for snn timesteps
snn_timesteps=50

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension)
print("Initial Weights:")
print(input_2_output_weight)

#Initialize an snn using the arguments defined above
snn = SNN(input_2_output_weight,
          input_dimension,
          output_dimension,
          v_decay,
          vth,
          snn_timesteps)

#Get the training data for AND gate using the function defined in 2a.
train_data = genORTrainData(snn_timesteps)

#Create an object of STDP class with appropriate arguments
stdp = STDP(
    network=snn,
    A_plus = 0.12,
    A_minus = 0.05,
    tau_plus = 12,
    tau_minus=5,
    lr=1e-5,
    snn_timesteps=snn_timesteps,
    epochs=75,
    w_min=0,
    w_max=1,
)

#Train the network using STDP
stdp.train(train_data)

#Test the trained network and print the network output for all 4 cases.
print("\nNetwork outputs after training:")
for data in train_data:
    input_spikes, target_output = data
    output = snn(input_spikes)
    print(f"Target: {target_output.mean()}, Predicted Output: {np.round(output)}, Actual Output: {output}")
print("\nFinal weights:")
print(snn.input_2_output_connection.weights)

Initial Weights:
[[0.5507979  0.70814782]]

Network outputs after training:
Target: 0.0, Predicted Output: [0.], Actual Output: [0.06]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]
Target: 1.0, Predicted Output: [1.], Actual Output: [0.92]

Final weights:
[[0.54926946 0.70661938]]
